In [1]:
from os.path import join, exists, dirname
from glob import glob
import pickle
import mplh.cluster_help as ch
import mplh.fig_utils as fu
import numpy as np
import os
import numpy as np
from scipy import sparse
from scipy.io import mmread
import matplotlib.pyplot as plt
from scipy.stats import hypergeom

import pandas as pd
import seaborn as sns
np.set_printoptions(formatter={'float': lambda x: format(x, '.5f')})

from svgutils.compose import *
import svgutils.transform as sg
import sys 

fig_utils from mplh


In [2]:
tmp_f = "/data/Mito_Trace/data/processed/figures/empty.svg"

In [24]:
# outdir = snakemake.params.outdir
# clone_var_dirs = snakemake.params.clone_var 
# clcounts_dirs = snakemake.params.clcounts
# dendro_dirs = snakemake.params.dendro 
# vaf_dirs = snakemake.params.vaf


indir = "/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/"
outdir = "/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_30/comparisons/"
donor = 0
gff_id = "gff_A2_black"
prm_variants = "variants_init"
prm_clone_k = "kparam_30"
# prms_enriched_barcode = ["objs_standard_constraints_None", "objs_standard_constraints_min_both_pct_0.5",
#                          "objs_high_unique_constraints_None", "objs_high_unique_constraints_min_both_pct_0.5"]
prms_enriched_barcode = "objs_standard_constraints_None;objs_standard_constraints_min_both_pct_0.5;objs_high_unique_constraints_None;objs_high_unique_constraints_min_both_pct_0.5"
prm_opt_top = "single_clones/donor0/cloneMethod_variants_init_knn_resolution_30/clonalShift_method_clones/top/"

#aggr_indir = "/data/Mito_Trace/output/aggregate/CHIP_aggr/v04/data/clone_var/variants.init_knnRes.30_objs_standard/aggregate/annotation/"

#variants_dirs = ["variants_init", "variants_simpleUnion", "variants_preFilterImpute"]



#enriched_barcode_weights = ["weights_standard", "weights_high_unique_None"]
#enriched_barcode_constraints = ["None", "both_0.5"]

#top_clones = ["top", "repr"]
#n_donors = 2



In [25]:
prms_enriched_barcode = prms_enriched_barcode.split("...")
prms_enriched_barcode

['objs_standard_constraints_None',
 'objs_standard_constraints_min_both_pct_0.5',
 'objs_high_unique_constraints_None',
 'objs_high_unique_constraints_min_both_pct_0.5']

In [4]:
method_names = {"clone":"knn"}
for i in prms_enriched_barcode:
    curr_obj, curr_cnstr =  i.split("constraints")
    curr_cnstr  = f"constraints {curr_cnstr.replace('_', ' ')}"
    curr_obj = curr_obj.split("objs_")[-1].strip("_")
    method_names[i] = f"{curr_obj}\n{curr_cnstr}"
    #method_names[i] = i.split("constraints")[-1]

In [5]:
method_names

{'clone': 'knn',
 'objs_standard_constraints_None': 'standard\nconstraints  None',
 'objs_standard_constraints_min_both_pct_0.5': 'standard\nconstraints  min both pct 0.5',
 'objs_high_unique_constraints_None': 'high_unique\nconstraints  None',
 'objs_high_unique_constraints_min_both_pct_0.5': 'high_unique\nconstraints  min both pct 0.5'}

In [6]:
clone_dir = join(indir, "clones", prm_variants, "knn", prm_clone_k)
dendro_dir = join(clone_dir, "barcodes/btwnClones_dendro_dt_dynamic")
anclo_dir = join(clone_dir, gff_id, "annotation_clones")
clone_count_dir = join(anclo_dir, "clone_counts")
top_dir = join(indir, prm_opt_top)
clone_dir

'/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_30'

In [7]:
dendro_dir

'/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_30/barcodes/btwnClones_dendro_dt_dynamic'

In [8]:
opt_dirs = {}
for i in prms_enriched_barcode:
    opt_dirs[i] = {}
    #opt_dirs[i]["clone_dir"] #clones/variants_init/knn/kparam_30/barcodes/btwnClones_dendro
    opt_dirs[i]["clone_dir"] = join(indir, "enriched_barcodes", i, "clones", prm_variants, "knn", prm_clone_k)
    opt_dirs[i]["dendro_dir"] = join(opt_dirs[i]["clone_dir"], "barcodes/btwnClones_dendro")
    
    opt_dirs[i]["top"] = join(indir, "enriched_barcodes", i, prm_opt_top)
    
    opt_dirs[i]["anclo_dir"] = join(opt_dirs[i]["clone_dir"], f"annotation_clones/")
    opt_dirs[i]["clone_count_dir"] = join(opt_dirs[i]["anclo_dir"], f"clone_counts/donor{donor}/")
    opt_dirs[i]["clone_dir"]
opt_dirs[i]

{'clone_dir': '/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/enriched_barcodes/objs_high_unique_constraints_min_both_pct_0.5/clones/variants_init/knn/kparam_30',
 'dendro_dir': '/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/enriched_barcodes/objs_high_unique_constraints_min_both_pct_0.5/clones/variants_init/knn/kparam_30/barcodes/btwnClones_dendro',
 'top': '/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/enriched_barcodes/objs_high_unique_constraints_min_both_pct_0.5/single_clones/donor0/cloneMethod_variants_init_knn_resolution_30/clonalShift_method_clones/top/',
 'anclo_dir': '/data/Mito_Trace/output/pipeline/v04/C

In [9]:

"""
based on https://neuroscience.telenczuk.pl/?p=331
"""

import svgutils.transform as sg


class Svg(object):
    "svg files with a data object (the svg), width, height and coordinates"

    def __init__(self, data, dim, coords, name, global_scale=0.5):
        self.data = data
        self.width = dim[0]*global_scale
        self.height = dim[1]*global_scale
        self.x = coords[0]
        self.y = coords[1]
        self.global_scale = global_scale
        self.name = name

    def scale_width_to_reference(self, reference_width):
        """Proportionally scale the image to a given width."""
        scalings_factor = reference_width / self.width
        self.data.moveto(0, 0)#, scale=scalings_factor)
        self.width = self.width * scalings_factor
        self.height = self.height * scalings_factor

    def scale_height_to_reference(self, reference_height):
        """Proportionally scale the image to a given height."""
        scalings_factor = reference_height / self.height
        self.data.moveto(0, 0)#, scale=scalings_factor)
        self.width = self.width * scalings_factor
        self.height = self.height * scalings_factor
        
    def scale_by_factor(self, scalings_factor):
        """Proportionally scale image by a scaling factor."""
        self.data.moveto(0, 0, scale=scalings_factor)
        self.width = self.width * scalings_factor
        self.height = self.height * scalings_factor

    def move(self, x, y):
        """Move the coordinates of an image."""
        self.data.moveto(x, y)
        self.x = x
        self.y = y


def get_size(svg_file):
    """Naively parse the svg text file to get the width and height."""
    with open(svg_file) as svg:
        for line in svg:
            if line.startswith('<svg'):
                paramdict = {e.split('="')[0]: e.split('="')[1] for e in line[5:-2].split('" ')}
                break
    print(paramdict["width"])
    print("height")
    print(paramdict["height"])
    return int(np.ceil(float(paramdict["width"].replace('pt', '').replace('"', '')))), int(np.ceil(float(paramdict["height"].replace('pt', '').replace('"', ''))))



def rescale_height(svgs, curr_keys, ref_key):
    """Change the dimensions of the images to the desired combinations."""
    for key in curr_keys:
        svgs[key].scale_height_to_reference(svgs[ref_key].height)
    #assert svgs[ref_key].height == svgs[curr_keys[0]].height == svgs[curr_keys[-1]].height


def rescale_width(svgs, curr_keys, ref_key):
    """Change the dimensions of the images to the desired combinations."""
    for key in curr_keys:
        svgs[key].scale_width_to_reference(svgs[ref_key].width)
    #assert svgs[ref_key].height == svgs[curr_keys[0]].height == svgs[curr_keys[-1]].height


def change_positions(df, svgs, widths, heights):
    """Move the images to the desired positions."""
    for i_row, row in enumerate(df.index):
        for i_col, col in enumerate(df.columns):
            curr = df.loc[row, col].split(".svg")[0]
            svgs[curr].move(sum(widths[:i_col]), sum(heights[:i_row]))
    

def letter_annotations(svgs, df=None, only_grid_titles=True, figure_names=None, transposed=False):
    if only_grid_titles:
        all_texts = []
        if transposed: # make donors the columns and figure_names the rows
            for ind in df.index:
                nm = figure_names[ind]
                value = svgs[df.loc[ind].iloc[0].split(".svg")[0]]
                all_texts.append(sg.TextElement(value.x, value.y, nm, size=10, weight="bold")) 
            for col in df.columns:
                value = svgs[df[col].iloc[0].split(".svg")[0]]
                all_texts.append(sg.TextElement(value.x, value.y-20, col, size=10, weight="bold"))
                
        else:
            for ind in df.index:
                value = svgs[df.loc[ind].iloc[0].split(".svg")[0]]
                all_texts.append(sg.TextElement(value.x, value.y, ind, size=10, weight="bold"))
            for col in df.columns:
                nm = figure_names[col]
                value = svgs[df[col].iloc[0].split(".svg")[0]]
                all_texts.append(sg.TextElement(value.x, value.y-20, nm, size=10, weight="bold"))    
        return all_texts
    """Add letters based on the location of the images."""
    return [sg.TextElement(value.x, value.y, value.name, size=10, weight="bold")
            for key, value in svgs.items()]
#     return [sg.TextElement(value.x + 10, value.y + 15, value.name, size=10, weight="bold")
#             for key, value in svgs.items()]


def files_to_svg_dict(df, scale=0.5, verbose=False):
    """Convert a list of images to a dictionary.
    Mapping the image basename to the Svg class instance,
    setting the dimensions based on sizes and coordinates (0,0) by default
    """
    out = {}
    for row in df.index:
        for col in df.columns:
            s = df.loc[row, col]
            key = df.loc[row, col].split(".svg")[0]
            name = f"{row} {col.replace('dendro', '').replace('.svg', '')}"
            if verbose:
                print(s)
            if not exists(s):
                s = tmp_f
            out[key] = Svg(
                data=sg.fromfile(s).getroot().scale(scale),
                dim=get_size(s),
                coords=(0, 0),
                global_scale=scale,
                name=name)            
    return out




def main_df(df, out_f, scale=0.25, figure_names=None, to_pdf=False):
    svgs = files_to_svg_dict(df, scale=scale)#df.values().flatten())
    
    for don, val in df.iterrows():
        curr_init_files = []
        curr_ref = val.iloc[0].split('.svg')[0] #svgs[val.iloc[0].split('.svg')[0]]
        curr_keys = [x.split('.svg')[0] for x in val.values] 
        rescale_height(svgs, curr_keys, curr_ref)
    
    col_max_widths = []

    for col in df.columns:
        curr_col_max = 0
        for ind in df.index:
            tmp_width = svgs[df.loc[ind, col].split(".svg")[0]].width
            if tmp_width > curr_col_max:
                curr_col_max = tmp_width
        col_max_widths.append(curr_col_max)

    row_max_heights = [] #should be the same in theory  
    for ind in df.index:
        curr_row_max = 0
        for col in df.columns:
            tmp_height = svgs[df.loc[ind, col].split(".svg")[0]].height
            if tmp_height > curr_row_max:
                curr_row_max = tmp_height
        row_max_heights.append(curr_row_max)
    
    change_positions(df, svgs, col_max_widths, row_max_heights)
    
    full_width = sum(col_max_widths) #[svgs[i].width for i in ['A', 'E']])
    full_height = sum(row_max_heights) #[svgs[i].height for i in ['A', 'B', 'C', 'D']])
    fig = sg.SVGFigure(full_width+20, full_height+20)
    text = letter_annotations(svgs, df=df, figure_names=figure_names)
    fig.append([s.data for s in svgs.values()])
    fig.append(text)
    fig.save(out_f)
 
    cmd = f'inkscape --file={out_f} -d 300 --export-area-drawing --without-gui --export-png={out_f.replace(".svg", ".png")}' # --export-area-drawing --without-gui --export-pdf=output.pdf'
    print(cmd)
    !{cmd}
    
    if to_pdf:
        cmd = f'inkscape --file={out_f} --export-area-drawing --without-gui --export-pdf={out_f.replace("svg", "pdf")} '
        print(cmd)
        !{cmd}
    
    

def main_df_transpose(df, out_f, scale=0.25, figure_names=None, to_pdf=False, to_transpose=True):
    
    svgs = files_to_svg_dict(df, scale=scale)#df.values().flatten())
    
    df = df.transpose()
    for don, val in df.iterrows():
        curr_init_files = []
        tmp = 0
        loop = True
        while loop and tmp <= len(val):
            if val.iloc[tmp] is not None:
                loop = False
            tmp += 1
        curr_ref = val.iloc[tmp].split('.svg')[0] #svgs[val.iloc[0].split('.svg')[0]]
        curr_keys = [x.split('.svg')[0] for x in val.values] 
        rescale_width(svgs, curr_keys, curr_ref)
    
    col_max_widths = []
    for col in df.columns:
        curr_col_max = 0
        for ind in df.index:
            if df.loc[ind,col] is not None:
                tmp_width = svgs[df.loc[ind, col].split(".svg")[0]].width
                if tmp_width > curr_col_max:
                    curr_col_max = tmp_width
        col_max_widths.append(curr_col_max)

    row_max_heights = [] #should be the same in theory  
    for ind in df.index:
        curr_row_max = 0
        for col in df.columns:
            tmp_height = svgs[df.loc[ind, col].split(".svg")[0]].height
            if tmp_height > curr_row_max:
                curr_row_max = tmp_height
        row_max_heights.append(curr_row_max)
    
    change_positions(df, svgs, col_max_widths, row_max_heights)
    
    full_width = sum(col_max_widths) #[svgs[i].width for i in ['A', 'E']])
    full_height = sum(row_max_heights) #[svgs[i].height for i in ['A', 'B', 'C', 'D']])
    fig = sg.SVGFigure(full_width+20, full_height+20)
    text = letter_annotations(svgs, df=df, figure_names=figure_names, transposed=True)
    fig.append([s.data for s in svgs.values()])
    fig.append(text)
    fig.save(out_f)
 
    cmd = f'inkscape --file={out_f} -d 300 --export-area-drawing --without-gui --export-png={out_f.replace(".svg", ".png")}' # --export-area-drawing --without-gui --export-pdf=output.pdf'
    print(cmd)
    !{cmd}
    
    if to_pdf:
        cmd = f'inkscape --file={out_f} --export-area-drawing --without-gui --export-pdf={out_f.replace("svg", "pdf")} '
        print(cmd)
        !{cmd}
    


In [10]:
#enriched_barcodes/objs_high_cells/clones/variants_init/knn/kparam_30

In [11]:
method_names

{'clone': 'knn',
 'objs_standard_constraints_None': 'standard\nconstraints  None',
 'objs_standard_constraints_min_both_pct_0.5': 'standard\nconstraints  min both pct 0.5',
 'objs_high_unique_constraints_None': 'high_unique\nconstraints  None',
 'objs_high_unique_constraints_min_both_pct_0.5': 'high_unique\nconstraints  min both pct 0.5'}

### Dendro

In [12]:
dendro_files = [f"donor{donor}.dendro.NoCondition.DP.svg", f"donor{donor}.dendro.NoCondition.max2.AF.svg"]

dendro_file_names = {x: x.split(".svg")[0] for x in dendro_files}


all_dendro_files = pd.DataFrame(index = ["clone"] + list(opt_dirs.keys()), columns = dendro_files)

all_dendro_files.loc["clone"] = [join(dendro_dir,dendro_files[0]), join(dendro_dir, dendro_files[1])]

for i in opt_dirs:
    all_dendro_files.loc[i] = [join(opt_dirs[i]["dendro_dir"], dendro_files[0]), 
                            join(opt_dirs[i]["dendro_dir"], dendro_files[1])]

all_dendro_files.index = all_dendro_files.index.map(method_names)
all_dendro_files

,donor0.dendro.NoCondition.DP.svg,donor0.dendro.NoCondition.max2.AF.svg
knn,/data/Mito_Trace/output/pipeline/v04/CHIP_b1/M...,/data/Mito_Trace/output/pipeline/v04/CHIP_b1/M...
standard\nconstraints None,/data/Mito_Trace/output/pipeline/v04/CHIP_b1/M...,/data/Mito_Trace/output/pipeline/v04/CHIP_b1/M...
standard\nconstraints min both pct 0.5,/data/Mito_Trace/output/pipeline/v04/CHIP_b1/M...,/data/Mito_Trace/output/pipeline/v04/CHIP_b1/M...
high_unique\nconstraints None,/data/Mito_Trace/output/pipeline/v04/CHIP_b1/M...,/data/Mito_Trace/output/pipeline/v04/CHIP_b1/M...
high_unique\nconstraints min both pct 0.5,/data/Mito_Trace/output/pipeline/v04/CHIP_b1/M...,/data/Mito_Trace/output/pipeline/v04/CHIP_b1/M...


In [13]:
main_df_transpose(all_dendro_files, join(outdir, "clones_dendro.svg"), scale=0.25, figure_names=dendro_file_names)

720pt
height
720pt
720pt
height
720pt
720pt
height
720pt
720pt
height
720pt
720pt
height
720pt
720pt
height
720pt
720pt
height
720pt
720pt
height
720pt
720pt
height
720pt
720pt
height
720pt
inkscape --file=/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_30/comparisons/clones_dendro.svg -d 300 --export-area-drawing --without-gui --export-png=/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_30/comparisons/clones_dendro.png
DPI: 300
Background RRGGBBAA: ffffff00
Area -2.39906:-260:947.861:127.598 exported to 2970 x 1211 pixels (300 dpi)
Bitmap saved as: /data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.

## VAF

In [14]:
var_files = ["top_variants_af.svg", "top_variants_distinct_pct_vs_other.svg"]
vaf_file_names = {x: x.split(".svg")[0] for x in var_files}

all_vaf_files = pd.DataFrame(index = ["clone"] + list(opt_dirs.keys()), columns = var_files)

all_vaf_files.loc["clone"] = [join(top_dir,"top_variants_af.svg"), join(top_dir, "top_variants_distinct_pct_vs_other.svg")]

for i in opt_dirs:
    all_vaf_files.loc[i] = [join(opt_dirs[i]["top"],"top_variants_af.svg"), 
                        join(opt_dirs[i]["top"],"top_variants_distinct_pct_vs_other.svg")]
       
#join(top_dir, "top_variants_af.svg")
#join(top_dir, "top_variants_distinct_pct_vs_other.svg")
all_vaf_files
all_vaf_files.index = all_vaf_files.index.map(method_names)

main_df_transpose(all_vaf_files, 
                  join(outdir, "clones_barcodes.svg"), scale=0.25, figure_names=vaf_file_names)

1664.921875
height
1500.6875"
626.040625pt
height
2998.414062pt
1664.83711
height
1501.7203125"
626.040625pt
height
1235.854062pt
1656.8617199999999
height
1501.7203125"
626.040625pt
height
1529.614062pt
1664.83711
height
1501.7203125"
626.040625pt
height
1235.854062pt
1656.8617199999999
height
1501.7203125"
626.040625pt
height
1529.614062pt
inkscape --file=/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_30/comparisons/clones_barcodes.svg -d 300 --export-area-drawing --without-gui --export-png=/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_30/comparisons/clones_barcodes.png
DPI: 300
Background RRGGBBAA: ffffff00
Area 0:-1026.92:2079.22:127.598 exported to 6498 x 3608 pixels (300

### Clone count

In [15]:
clcount_file_names =  {
    "normalized_counts.barplot_conditions.svg": "normalized clone counts",
    "clone_counts.barplot_conditions.svg": "clone sizes",
    "clone_counts.conditionScatter.svg": "scatterplot of ncells across conditions"
}

#clone_count_dir

clcount_files = list(clcount_file_names.keys()) #["top_clcountiants_af.svg", "top_clcountiants_distinct_pct_vs_other.svg"]
#clcount_file_names = {x: x.split(".svg")[0] for x in clcount_files}

all_clcount_files = pd.DataFrame(index = ["clone"] + list(opt_dirs.keys()), columns = clcount_files)

all_clcount_files.loc["clone"] = [join(clone_count_dir, f"donor{donor}.normalized_clone_counts.barplot_conditions.svg"), 
                                  join(clone_count_dir, f"donor{donor}.{clcount_files[1]}"), 
                                  join(clone_count_dir, f"donor{donor}.{clcount_files[2]}")]
#donor0.normalized_clone_counts.barplot_conditions
for i in opt_dirs:
    all_clcount_files.loc[i] = [join(opt_dirs[i]["clone_count_dir"], clcount_files[0]), 
                                join(opt_dirs[i]["clone_count_dir"], clcount_files[1]),
                                join(opt_dirs[i]["clone_count_dir"], clcount_files[2])]
       
#join(top_dir, "top_clcountiants_af.svg")
#join(top_dir, "top_clcountiants_distinct_pct_vs_other.svg")

all_clcount_files.index = all_clcount_files.index.map(method_names)

main_df_transpose(all_clcount_files, 
                  join(outdir, "clones_counts.svg"), scale=0.25, figure_names=clcount_file_names)

864pt
height
864pt
864pt
height
864pt
864pt
height
864pt
864pt
height
864pt
864pt
height
864pt
864pt
height
864pt
864pt
height
864pt
864pt
height
864pt
864pt
height
864pt
864pt
height
864pt
864pt
height
864pt
864pt
height
864pt
864pt
height
864pt
864pt
height
864pt
864pt
height
864pt
inkscape --file=/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_30/comparisons/clones_counts.svg -d 300 --export-area-drawing --without-gui --export-png=/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_30/comparisons/clones_counts.png
DPI: 300
Background RRGGBBAA: ffffff00
Area 0:-548:1091.86:127.598 exported to 3412 x 2111 pixels (300 dpi)
Bitmap saved as: /data/Mito_Trace/output/pipeline/v04/CHIP_b1

In [16]:
all_clcount_files.iloc[0,0]

'/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_30/gff_A2_black/annotation_clones/clone_counts/donor0.normalized_clone_counts.barplot_conditions.svg'

In [17]:
clcount_files

['normalized_counts.barplot_conditions.svg',
 'clone_counts.barplot_conditions.svg',
 'clone_counts.conditionScatter.svg']

In [18]:
all_clcount_files.iloc[0,0]

'/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_30/gff_A2_black/annotation_clones/clone_counts/donor0.normalized_clone_counts.barplot_conditions.svg'

In [19]:
all_clcount_files.iloc[2,0]

'/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/enriched_barcodes/objs_standard_constraints_min_both_pct_0.5/clones/variants_init/knn/kparam_30/annotation_clones/clone_counts/donor0/normalized_counts.barplot_conditions.svg'

In [20]:
clone_count_dir

'/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_30/gff_A2_black/annotation_clones/clone_counts'

In [21]:
all_clcount_files

,normalized_counts.barplot_conditions.svg,clone_counts.barplot_conditions.svg,clone_counts.conditionScatter.svg
knn,/data/Mito_Trace/output/pipeline/v04/CHIP_b1/M...,/data/Mito_Trace/output/pipeline/v04/CHIP_b1/M...,/data/Mito_Trace/output/pipeline/v04/CHIP_b1/M...
standard\nconstraints None,/data/Mito_Trace/output/pipeline/v04/CHIP_b1/M...,/data/Mito_Trace/output/pipeline/v04/CHIP_b1/M...,/data/Mito_Trace/output/pipeline/v04/CHIP_b1/M...
standard\nconstraints min both pct 0.5,/data/Mito_Trace/output/pipeline/v04/CHIP_b1/M...,/data/Mito_Trace/output/pipeline/v04/CHIP_b1/M...,/data/Mito_Trace/output/pipeline/v04/CHIP_b1/M...
high_unique\nconstraints None,/data/Mito_Trace/output/pipeline/v04/CHIP_b1/M...,/data/Mito_Trace/output/pipeline/v04/CHIP_b1/M...,/data/Mito_Trace/output/pipeline/v04/CHIP_b1/M...
high_unique\nconstraints min both pct 0.5,/data/Mito_Trace/output/pipeline/v04/CHIP_b1/M...,/data/Mito_Trace/output/pipeline/v04/CHIP_b1/M...,/data/Mito_Trace/output/pipeline/v04/CHIP_b1/M...


### All folders needed for one donor to compare methods:
1. After clone-calling:
A. Dendrogram (dynamic)
B. Clone count barplots
Scatterplot
C. VAF 
D. lineage clone count
E. MT as clones clone count 
F. MT as clones clone distribution
G. Dynamic dendro lineage clone count 
H. Dynamic dendro lineage VAF

2. After enriched barcode:
A-D as above, plus:
E. multi-objective barcode analysis


Additionally, parameters to use:
1. Clone-calling:
- init
- prefilterImpute
- simpleUnion
2. Multi-objective parameters:
- standard_None
- standard_min_both_pct_0.5
- high_unique_None
- high_unique_min_both_pct_0.5
3. Clone detection parameters:
- Clones
- Dynamic dendro
- MT as clones
- MT as clones dendro


Goal is to compare across all the methods the results visually.
We want to compare across clone-calling methods and before and after enriched barcodes. 



In [22]:
# indir

# vaf_figure_names =  {
#     "top_variants_af.svg":"vaf",
#     "top_variants_distinct_pct_vs_other.svg": "barcode pct in clone"
# }
# dendro_figure_names =  {"dendro.NoCondition.max2.AF.svg": "AF (max 0.2)",
#              "dendro.NoCondition.max5.AF.svg": "AF (max 0.5)",
#              "dendro.NoCondition.DP.svg": "Log2 reads per cell-MT position"}

# clonevar_figure_names = {
#     "best_params.svg": "Final clone-barcode table",
#     "top10perc_param_group_clusters.svg": "Cluster top hyperparameters by multi-objectives",
#     "elbow_multi_obj_nclust.svg": "Elbow plot for determining number of objective clusters",
#     "multi_clusters_nclust.svg": "multi_obj_clusters" ,
#     "top_param_group_results.svg": "top_objective_scores"
#     #"top_param_groups_clone_vars.svg": "clone-barcode table of top hyperparameter clusters",
# }


# cl_count_figure_names =  {
#     "normalized_counts.barplot_conditions.svg": "normalized clone counts",
#     "top20_minCell10_clone_counts.barplot_conditions.svg": "top 20 clones",
#     "clone_counts.conditionScatter.svg": "scatterplot of ncells across conditions"
# }

In [23]:
#"single_clones/donor0/cloneMethod_variants_init_knn_resolution_30/clonalShift_method_clones/top/"